<a href="https://colab.research.google.com/github/MachineNeyarning/ClassifiersCommittee/blob/main/ClassifiersCommittee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comitê de Classificadores | Projeto N1

**GitHub: https://github.com/MachineNeyarning/ClassifiersCommittee**

---

## Grupo:
- Gustavo Henrique Martins
- Ícaro Botelho
- Maria Clara Seixa Scheffel
- Maruan Biasi El Achkar
- Ricardo Falcão Schlieper

### Dataset escolhido: Rain in Australia
- Detalhamento de colunas em: https://github.com/MachineNeyarning/ClassifiersCommittee/tree/main/dataset
- Fonte: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package
- Target: RainTomorrow

### Algoritmos escolhidos:
- K-Nearest Neighbors (KNN)
- Decision Tree Classifier (Tree-Based)
- Random Forest Classifier (Tree-Based)
- LightGBM Classifier (Tree-Based)
- CatBoost Classifier (Tree-Based)
- Gaussian Naive Bayes (Naive Bayes)
- Support Vector Classification (SVC)
- Multi-Layer Perceptron (MLPClassifier)
- AdaBoost Classifier (Ensemble Methods)
- Quadratic Discriminant Analysis (QDA)



---
# Preparar Dataset

In [1]:
# Baixar Dataset

import kagglehub
import os

path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")
csv_path = os.path.join(path, 'weatherAUS.csv')


Using Colab cache for faster access to the 'weather-dataset-rattle-package' dataset.


In [ ]:
# Criar Dataframe Pandas

import pandas as pd

df = pd.read_csv(csv_path, sep=',', engine='python', on_bad_lines='skip')
df.head(3)

In [ ]:
# Describe Dataframe Data

df.describe()

In [ ]:
# Verificar falta de dados

missing_data = df.isnull().sum()
display(missing_data)

In [ ]:
# Remover linhas com dados faltando

df_cleaned = df.dropna()
display(df_cleaned.head())
print(f"Original: {df.shape}")
print(f"Limpo: {df_cleaned.shape}")

---
# 12 Algoritimos:

---
## 1) K-Nearest Neighbors (KNN) Falcão

In [ ]:
# Copia do dataframe
df_knn = df_cleaned

###Importar as Bibliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
# Excluir a coluna 'Date' se não for relevante
df_knn = df_knn.drop(columns=['Date'])

# Converter variáveis categóricas em variáveis dummy (one-hot encoding)
df_knn = pd.get_dummies(df_knn, drop_first=True)

# Separar variáveis independentes (X) e dependentes (y)
X = df_knn.drop(columns=['RainTomorrow_Yes'])  # Substitua 'PlayTennis_Yes' pelo nome da coluna alvo
y = df_knn['RainTomorrow_Yes']

In [ ]:
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Inicializar e treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=5) # Você pode ajustar o número de vizinhos
knn.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred = knn.predict(X_test_scaled)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"KNN Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(report)

---
## 2) Decision Tree Classifier (Tree-Based) | Gustavo

In [ ]:
# Copia do dataframe
df_dtc = df_cleaned

In [ ]:
# ========================
# Decision Tree Classifier
# ========================
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# features (X) e target (y)
X = df_dtc.drop(["RainTomorrow", 'Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], axis=1)  # variável alvo = "RainTomorrow"
y = df_dtc["RainTomorrow"]

# converter TARGET para numerico
le = LabelEncoder()
y = le.fit_transform(y)

# treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modelo e prediçoes
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)

# métricas
print("Decision Tree")
print("Acurácia:", accuracy_score(y_test, y_pred_dtc))
print(confusion_matrix(y_test, y_pred_dtc))
print(classification_report(y_test, y_pred_dtc))

# ROC Curve
y_prob_dtc = dtc.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, y_prob_dtc)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'Decision Tree (AUC = {roc_auc:.2f})')

# Keep the plot code from the Random Forest cell to display all ROC curves together
# plt.plot([0,1], [0,1], "k--")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("ROC Curve - Tree Based Models")
# plt.legend()
# plt.show()

---
## 3) Random Forest Classifier (Tree-Based) | Gustavo

In [ ]:
# Copia do dataframe
df_rfc = df_cleaned

In [ ]:
# ========================
# Random Forest Classifier
# ========================
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# features (X) e target (y)
X = df_rfc.drop(["RainTomorrow", 'Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'], axis=1)
y = df_rfc["RainTomorrow"]

# Convert TARGET to numeric
le = LabelEncoder()
y = le.fit_transform(y)

# treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# modelo e prediçoes
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

# métricas
print("Random Forest")
print("Acurácia:", accuracy_score(y_test, y_pred_rfc))
print(confusion_matrix(y_test, y_pred_rfc))
print(classification_report(y_test, y_pred_rfc))

# ROC Curve
y_prob_rfc = rfc.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, y_prob_rfc)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'Random Forest (AUC = {roc_auc:.2f})')

# gráfico ROC
plt.plot([0,1], [0,1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Tree Based Models")
plt.legend()
plt.show()

---
## 4) XGBoost (Tree-Based) | Maruan

In [ ]:
# Copia do dataframe
df_xgb = df_cleaned

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_xgb = df_xgb.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_xgb = df_xgb['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_xgb = le.fit_transform(y_xgb)

In [ ]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_xgb, y_xgb, test_size=0.2, random_state=42, stratify=y_xgb)

In [ ]:
# XGBoost model

# resultado com parametros padrao: 85.69%
xgboost = xgb.XGBClassifier(random_state=42, eval_metric='logloss')

xgboost.fit(X_train_xgb, y_train_xgb)

In [ ]:
# Avaliacao

y_pred_xgb = xgboost.predict(X_test_xgb)
accuracy_xgb = accuracy_score(y_test_xgb, y_pred_xgb)

print(f"XGBoost Accuracy: {accuracy_xgb * 100:.2f}%")

---
## 5) LightGBM Classifier (Tree-Based) | Maruan

In [ ]:
# Copia do dataframe
df_lgbm = df_cleaned

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_lgbm = df_lgbm.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_lgbm = df_lgbm['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_lgbm = le.fit_transform(y_lgbm)

In [ ]:
X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(X_lgbm, y_lgbm, test_size=0.2, random_state=42, stratify=y_lgbm)

In [ ]:
# LightGBM model

# resultado com parametros padrao: 85.90%
# melhor resultado com parametros personalizados: 86.46%
lgbm = lgb.LGBMClassifier(
    random_state=42,
    n_estimators=52783,
    learning_rate=0.055,
    num_leaves=80,
    max_depth=-1,
    max_bin=716,
    min_data_in_leaf=60,
    min_sum_hessian_in_leaf=5.0,
    min_gain_to_split=0.01,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    lambda_l1=0.1,
    lambda_l2=2.0,
    # extra_trees=True,
    # class_weight="balanced",
    n_jobs=-1,
    verbose=-1 # para nao encher de log no console senao o colab fica com 500mb
)

lgbm.fit(
    X_train_lgbm, y_train_lgbm,
    eval_set=[(X_test_lgbm, y_test_lgbm)],
    eval_metric=["auc", "binary_logloss"],
    callbacks=[lgb.early_stopping(100)]
)

In [ ]:
# Avaliacao
y_pred_lgbm = lgbm.predict(X_test_lgbm)
accuracy_lgbm = accuracy_score(y_test_lgbm, y_pred_lgbm)

print(f"LightGBM Accuracy: {accuracy_lgbm * 100:.2f}%")

---
## 6) CatBoost Classifier (Tree-Based) | Maruan

In [ ]:
# Copia do dataframe
df_cb = df_cleaned

In [ ]:
!pip install catboost

import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_cb = df_cb.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_cb = df_cb['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_cb = le.fit_transform(y_cb)

In [ ]:
X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(X_cb, y_cb, test_size=0.2, random_state=42, stratify=y_cb)

In [ ]:
# CatBoost Model

# resultado com parametros padrao: 86.46%
# melhor resultado com parametros personalizados: 86.37%

catboost = cb.CatBoostClassifier(
    random_state=40,
    # iterations=5000,
    # learning_rate=0.1,
    # depth=6,
    # l2_leaf_reg=3,
    # model_size_reg=None,
    # colsample_bylevel=0.8,
    # loss_function='Logloss',
    # eval_metric='Accuracy',
    # leaf_estimation_iterations=10,
    # bootstrap_type='Bayesian',
    # bagging_temperature=0.3,
    # random_strength=0.2,
    # auto_class_weights="Balanced",
    # od_type='Iter',
    # od_wait=50,
    verbose=0,
)

catboost.fit(X_train_cb, y_train_cb)

In [ ]:
# Avaliacao
y_pred_cb = catboost.predict(X_test_cb)
accuracy_cb = accuracy_score(y_test_cb, y_pred_cb)

print(f"CatBoost Accuracy: {accuracy_cb * 100:.2f}%")

---
## 7) Perceptron | Maruan

In [ ]:
# Copia do dataframe
df_perceptron = df_cleaned

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Remover tabelas nao numericas e o target
X_perceptron = df_perceptron.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_perceptron = df_perceptron['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_perceptron = le.fit_transform(y_perceptron)

In [ ]:
X_train_perceptron, X_test_perceptron, y_train_perceptron, y_test_perceptron = train_test_split(X_perceptron, y_perceptron, test_size=0.2, random_state=42, stratify=y_perceptron)

In [ ]:
# escalar
scaler = StandardScaler()
X_train_scaled_perceptron = scaler.fit_transform(X_train_perceptron)
X_test_scaled_perceptron = scaler.transform(X_test_perceptron)

In [ ]:
# Perceptron model

perceptron = Perceptron(random_state=43)

perceptron.fit(X_train_scaled_perceptron, y_train_perceptron)

In [ ]:
# Avaliacao

y_pred_perceptron = perceptron.predict(X_test_scaled_perceptron)

accuracy_perceptron = accuracy_score(y_test_perceptron, y_pred_perceptron)

print(f"Perceptron Accuracy: {accuracy_perceptron * 100:.2f}%")

---
## 8) Gaussian Naive Bayes (Naive Bayes) Falcão

In [ ]:
# Copia do dataframe
df_gnb = df_cleaned

### Conferir Data

In [ ]:
if 'Date' in df_gnb.columns:
    df_gnb = df_gnb.drop(columns=['Date'])

### Transformar colunas categóricas em números

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

### # Separar X e y (ajuste o nome da coluna alvo)

In [ ]:
# Separar X e y (ajuste o nome da coluna alvo)
X = df_gnb.drop(columns=['RainTomorrow'])
y = df_gnb['RainTomorrow']

# Converter variáveis categóricas em variáveis dummy (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

### Dividir e treinar

In [ ]:
# Separar X e y (ajuste o nome da coluna alvo)
X = df_gnb.drop(columns=['RainTomorrow']) # Drop the 'Date' column
y = df_gnb['RainTomorrow']

# Converter variáveis categóricas em variáveis dummy (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Escalonar variáveis

In [ ]:
from sklearn.preprocessing import StandardScaler

# Escalonar variáveis
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Treinar Gaussian Naive Bayes

In [ ]:
# Converter variáveis categóricas em variáveis dummy (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

In [ ]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Ensure the data is in a suitable format (e.g., NumPy array with float type)
X_train = np.asarray(X_train, dtype=np.float64)
X_test = np.asarray(X_test, dtype=np.float64)


gnb = GaussianNB()
gnb.fit(X_train, y_train)

In [ ]:
print("Missing values in X_train after preprocessing:")
print(pd.DataFrame(X_train).isnull().sum().sum())

print("Missing values in X_test after preprocessing:")
print(pd.DataFrame(X_test).isnull().sum().sum())

### Avaliar

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = gnb.predict(X_test)
print(f'Acurácia: {accuracy_score(y_test, y_pred):.2f}')
print('Relatório de Classificação:')
print(classification_report(y_test, y_pred))

### tentar melhorar a acurácia

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE  # Se tiver classes desbalanceadas

# 1. Balancear dados (se necessário)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 2. Selecionar melhores features (por exemplo as 10 melhores)
selector = SelectKBest(f_classif, k=10)
X_selected = selector.fit_transform(X_resampled, y_resampled)

# 3. Dividir os dados novamente
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.3, random_state=42)

# 4. Escalonar
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. Treinar GaussianNB com ajuste var_smoothing
gnb = GaussianNB(var_smoothing=1e-9)
gnb.fit(X_train, y_train)

# 6. Avaliar com cross-validation e teste final
scores = cross_val_score(gnb, X_train, y_train, cv=5)
print(f'Cross-validation accuracy média: {scores.mean():.2f}')

y_pred = gnb.predict(X_test)
print(f'Acurácia no teste: {accuracy_score(y_test, y_pred):.2f}')
print(classification_report(y_test, y_pred))


---
## 9) Support Vector Classification (SVC) | MC

In [ ]:
# Copia do dataframe
df_svc = df_cleaned

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# Separando Features e target
X = df_svc.drop(columns=[
    'RainTomorrow', 'Date', 'Location',
    'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'
])
y = df_svc['RainTomorrow']

In [ ]:
# Codificando Target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# Escalando
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Treinando svc
svc = SVC(kernel='rbf', C=1.0, random_state=42)
svc.fit(X_train_scaled, y_train)

In [ ]:
# Previsao
y_pred = svc.predict(X_test_scaled)

In [ ]:
# Avaliacao
accuracy = accuracy_score(y_test, y_pred)
print(f"\nSVC accuracy score {accuracy:.2f}\n")






---
## 10) Multi-Layer Perceptron (MLPClassifier) |

In [ ]:
# Copia do dataframe
df_mlp = df_cleaned

---
## 11) AdaBoost Classifier (Ensemble Methods) | MC

In [ ]:

# Copia do dataframe
df_ada = df_cleaned

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Separar features e target
X = df_ada.drop(columns=[
    'RainTomorrow', 'Date', 'Location',
    'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'
])
y = df_ada['RainTomorrow']

In [ ]:
# Codificando Target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Escalando
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Treinando ada
ada = AdaBoostClassifier(n_estimators=100, random_state=42)
ada.fit(X_train_scaled, y_train)

In [ ]:
# Previsao
y_pred_ada = ada.predict(X_test_scaled)
accuracy_ada = accuracy_score(y_test, y_pred_ada)

In [ ]:
# Avaliacao
print(f"AdaBoost Accuracy: {accuracy_ada * 100:.2f}%")

---
## 12) Quadratic Discriminant Analysis (QDA) |

In [ ]:
# Copia do dataframe
df_qda = df_cleaned

---
# Comite Classificador:

In [ ]:
# Copia do dataframe
df_comittee = df_cleaned

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Remover tabelas nao numericas e o target
X_committee = df_comittee.drop(['Date', 'Location', 'RainToday', 'RainTomorrow', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
y_committee = df_comittee['RainTomorrow']

# Converter TARGET para numerico
le = LabelEncoder()
y_committee = le.fit_transform(y_committee)

In [ ]:
# mesma coisa dos classifiers mas para o comite
X_train_committee, X_test_committee, y_train_committee, y_test_committee = train_test_split(X_committee, y_committee, test_size=0.2, random_state=42, stratify=y_committee)

In [ ]:
# QUAIS CLASSIFICADORES VAO SER USADOS NO COMITE - QUANDO TERMINAREM UM ALGORITIMO NOVO, COLOCA ELE AQUI !!!!!!!!!!!!!
estimators = [('lgbm', lgbm), ('catboost', catboost), ('xgboost', xgboost), ('perceptron', perceptron), ('svc', svc), ('adaboost', ada)]

In [ ]:
# Classifiers Comittee

# Tipos de voting
# 'hard' = maioria
# 'soft' usa pesos, weights=[]
weights_acc = [accuracy_lgbm, accuracy_cb] # pesos sendo a acurracy de cada modelo

committee = VotingClassifier(estimators=estimators, voting='hard')


committee.fit(X_train_committee, y_train_committee)

In [ ]:
# Avaliacao

from sklearn.metrics import f1_score

# RESULTADOS COM APENAS LightGBM e CatBoost (maruan)
# Acurracy com HARD VOTING: 85.97%
# Acurracy com SOFT e accuracy dos algoritimos como weights: 85.86%
# tentei fazer um grid search pelos melhores pesos mas rodou por 6 horas e crashou
# cada vez que eu boto um algoritimo novo fica pior KKKKKKKKKKKKKKKKKKKK

accuracy_committee = committee.score(X_test_committee, y_test_committee)
y_pred_committee = committee.predict(X_test_committee)
accuracyf1_committee = f1_score(y_test_committee, y_pred_committee)


print(f"Classifier Committee Accuracy: {accuracy_committee * 100:.2f}%")
print(f"Classifier Committee F1-score: {accuracyf1_committee * 100:.2f}%")

print('------------------------------------------')
algorithm_names = [name for name, _ in estimators]
print(f"Algoritmos do Comite: {algorithm_names}")


---
#### sugestao de outros algoritimos:


*   Logistic Regression
*   SGDClassifier
*   Passive Aggressive Classifier (nao sei se funcionaria aqui)
*   Extra Trees Classifier (tipo um random forest ao quadrado)
*   GradientBoostingClassifier
*   HistGradientBoostingClassifier (uma versao customizada do XGBoost/LightGBM)
*   Linear Discriminant Analysis
*   LogitBoost
*   Bagged Perceptrons
*   AdaBoost com perceptrons dentro

